# Mobile App for Lottery Addiction

Many people start playing the lottery for fun, but for some this activity turns into a habit which eventually escalates into addiction. Like other compulsive gamblers, lottery addicts soon begin spending from their savings and loans, they start to accumulate debts, and eventually engage in desperate behaviors like theft.

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the [6/49 lottery](https://en.wikipedia.org/wiki/Lotto_6/49) and build functions that enable users to answer questions like:

- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

The institute also wants us to consider historical data coming from the national 6/49 lottery game in Canada. The [data set](https://www.kaggle.com/datascienceai/lottery-dataset) has data for 3,665 drawings, dating from 1982 to 2018.

## Preparing core functions

Since we will use these functions a lot, we start by writing functions to calculate factorials, permutations, and combinations.

In [1]:
# Function to calculate factorials
def factorial(n):
    if n == 0:
        return 1
    else:
        factors = []
        for i in range(n):
            factors.append(n - i)
        product = 1
        for factor in factors:
            product *= factor
        return product

# Test function
factorial(5)

120

In [2]:
# Function to calculate permutations (order matters - Ordered sampling without replacement)
def permutations(n, k):
    if k > n:
        return 0
    else:
        return factorial(n) / factorial(n - k)

# Test function
permutations(3, 2)

6.0

In [3]:
# Function to calculate combinations (order doesn't matter - Unordered sampling without replacement)
def combinations(n, k):
    return permutations(n, k) / factorial(k)

# Test function
combinations(49, 6)

13983816.0

## One-ticket probability

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn. If a player has a ticket with the numbers {13, 22, 24, 27, 42, 44}, he only wins the big prize if the numbers drawn are {13, 22, 24, 27, 42, 44}. If only one number differs, he doesn't win.

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

The function will take in 6 numbers and return a winning probability for the big prize, i.e. six correct numbers.

In [4]:
# One-ticket probability

ticket = [2, 3, 5, 8, 23, 40]
def one_ticket_probability(ticket):
    ticket = ticket
    combs = combinations(49, 6)
    num_success = 1
    prob_win = num_success / combs
    print ("Your chosen numbers are: {}.".format(ticket))
    message = "The winning probability for your ticket is: {:.7f} %.".format(prob_win * 100)
    return message
    
one_ticket_probability(ticket)    

Your chosen numbers are: [2, 3, 5, 8, 23, 40].


'The winning probability for your ticket is: 0.0000072 %.'

## Historical data check for Canada Lottery

For the first version of the app, however, users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now. The data set contains historical data for 3,665 drawings (each row shows data for a single drawing), dating from 1982 to 2018. 

Let's load and explore the data and write a function that compares the user's input with the historical data.


In [5]:
import pandas as pd
import numpy as np

lotto = pd.read_csv('649.csv')
lotto.shape

(3665, 11)

In [6]:
lotto.head(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34


In [7]:
lotto.tail(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


## Function for historical data check

Now we write the function for the data check. We first write a function that takes as input a rownumber from the dataset and returns the corresponding six winning numbers.

In [8]:
# Function to extract numbers
def extract_numbers(rownum):
    draw = set(lotto.iloc[rownum, 4:10])
    return draw

# Test function
extract_numbers(3663)
    

{2, 15, 21, 31, 38, 49}

Now we extract all numbers and write all the sets into a series object. But in order to make that work in combination with the `apply()` method, we need to not use explicit `iloc` indexing in the function. (Each row is automatically passed to the function when using `apply()`.)

In [9]:
# Function to extract numbers
def extract_numbers(rownum):
    draw = set(num for num in rownum[4:10])
    return draw

# Write all sets into a series
winning_draws = lotto.apply(extract_numbers, axis=1)
winning_draws.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
dtype: object

Next we will write the function that checks the list of six numbers supplied by the user against the extracted numbers.

In [10]:
# Function to check the historical occurrence of numbers
def check_historical_occurrence(ticket, winning_draws):
    ticket = set(ticket)
    match = ticket == winning_draws
    combs = combinations(49, 6)
    num_success = 1
    prob_win = num_success / combs
    num_occur = match.sum()
    print ("Your chosen numbers are: {}.".format(ticket))
    message_1 = "In past draws the numbers that you provided won the big prize {} times.".format(num_occur)
    message_2 = "The winning probability for your ticket in the next drawing is: {:.7f} %.".format(prob_win * 100)
    print(message_1)
    print(message_2)
    print("The chances of winning with your numbers remain the same, regardless of how many times a combination was drawn in the past.")
    
check_historical_occurrence(ticket, winning_draws)    

Your chosen numbers are: {2, 3, 5, 8, 40, 23}.
In past draws the numbers that you provided won the big prize 0 times.
The winning probability for your ticket in the next drawing is: 0.0000072 %.
The chances of winning with your numbers remain the same, regardless of how many times a combination was drawn in the past.


Since each lottery drawing is independent from previous drawings, the probability of winning the big prize at any given drawing remains the same: 0.0000072 %. We could in theory write the function in a simpler way, by just hard-coding that probability in. However, writing the function this way allows us to adjust the function further in later steps. For example to check the possibility of winning 3 out of 49, etc.

## Multi-ticket probability

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning — hence, we're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.
We've talked with the engineering team and they gave us the following information:

- The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
- Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
- The function should print information about the probability of winning the big prize depending on the number of different tickets played.

In [12]:
# Multi-ticket probability
def multi_ticket_probability(num_tickets):
    combs = combinations(49, 6)
    num_success = num_tickets
    prob_win = num_success / combs
    print("Number of tickets that you are playing: {}.".format(num_tickets))
    message = "The winning probability for playing this number of tickets is: {:.7f} %.".format(prob_win * 100)
    return message

ticket_nums = [1, 10, 100, 10000, 1000000, 6991908, 13983816]
for num_tickets in ticket_nums:
    output = multi_ticket_probability(num_tickets)
    print(output)

Number of tickets that you are playing: 1.
The winning probability for playing this number of tickets is: 0.0000072 %.
Number of tickets that you are playing: 10.
The winning probability for playing this number of tickets is: 0.0000715 %.
Number of tickets that you are playing: 100.
The winning probability for playing this number of tickets is: 0.0007151 %.
Number of tickets that you are playing: 10000.
The winning probability for playing this number of tickets is: 0.0715112 %.
Number of tickets that you are playing: 1000000.
The winning probability for playing this number of tickets is: 7.1511238 %.
Number of tickets that you are playing: 6991908.
The winning probability for playing this number of tickets is: 50.0000000 %.
Number of tickets that you are playing: 13983816.
The winning probability for playing this number of tickets is: 100.0000000 %.


When testing the function with several inputs, we can see that increasing the number of tickets barely changes the winning probabilities in a meaningful way. Even with 1 Million tickets played, the winning probability for the big prize is only a modest 7.15%. Playing 6,991,908 tickets will have a 50% of winning. Playing 13,983,816 tickets - one for each possible combination - will guarantee winning the jackpot. Just that the jackpot will never yield enough winnings to pay for the number of tickets needed. So essentially, if a lotto addict might think that he or she needs to just increase the number of tickets to win, then this will not matter for the probability unless that person buys a prohibitively high number of tickets.

## Writing a function for less than six winning numbers

In most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

These are the engineering details we'll need to be aware of:

- Inside the app, the user inputs:
    - six different numbers from 1 to 49; and
    - an integer between 2 and 5 that represents the number of winning numbers expected
- Our function prints information about the probability of having the inputted number of winning numbers.

To calculate the probabilities, we tell the engineering team that the specific combination on the ticket is irrelevant behind the scenes, and we only need the integer between 2 and 5 representing the number of winning numbers expected.
So we will code heare a function that just takes in the number of winning numbers expected and that will return the probability of getting exactly this many numbers correct.

In [13]:
# Function to calculate winning probabilities for less than six numbers
def probability_less_6(exp_win):
    combs = combinations(49, 6)
    pos_combs = combinations(6, exp_win)
    remaining_combs = combinations(43, 6 - exp_win)
    num_success = pos_combs * remaining_combs
    prob_win = num_success / combs
    print("Expected correct numbers: {}.".format(exp_win))
    message = "The winning probability for getting this many numbers right is: {:.7f} %.".format(prob_win * 100)
    return message

expected_wins = [1, 2, 3, 4, 5, 6]
for exp_wins in expected_wins:
    output = probability_less_6(exp_wins)
    print(output)

Expected correct numbers: 1.
The winning probability for getting this many numbers right is: 41.3019450 %.
Expected correct numbers: 2.
The winning probability for getting this many numbers right is: 13.2378029 %.
Expected correct numbers: 3.
The winning probability for getting this many numbers right is: 1.7650404 %.
Expected correct numbers: 4.
The winning probability for getting this many numbers right is: 0.0968620 %.
Expected correct numbers: 5.
The winning probability for getting this many numbers right is: 0.0018450 %.
Expected correct numbers: 6.
The winning probability for getting this many numbers right is: 0.0000072 %.


While we are basically only interested in the winning probabilities for the number 2, 3, 4, 5, our function can handle all inputs from 1 to 6. So just to get an idea what the probability is to get 1 or 6 numbers right, I added them here as well.

## Next steps

For the first version of the app, we coded four main functions:
- `one_ticket_probability()` — calculates the probability of winning the big prize with a single ticket
- `check_historical_occurrence()` — checks whether a certain combination has occurred in the Canada lottery data set
- `multi_ticket_probability()` — calculates the probability for any number of of tickets between 1 and 13,983,816
- `probability_less_6()` — calculates the probability of having two, three, four or five winning numbers exactly

Possible features for a second version of the app include:

- Making the outputs even easier to understand by adding fun analogies (for example, we can find probabilities for strange events and compare with the chances of winning in lottery; for instance, we can output something along the lines "You are 100 times more likely to be the victim of a shark attack than winning the lottery")
- Combining the one_ticket_probability() and check_historical_occurrence() to output information on probability and historical occurrence at the same time
- Create a function similar to probability_less_6() which calculates the probability of having at least two, three, four or five winning numbers. The number of successful outcomes for having at least four winning numbers is the sum of these three numbers:
    - The number of successful outcomes for having four winning numbers exactly
    - The number of successful outcomes for having five winning numbers exactly
    - The number of successful outcomes for having six winning numbers exactly